Florida Lotto Pick 6

In [1]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import tensorflow as tf
from collections import namedtuple
from collections import Counter
from random import randint
from IPython.core.debugger import Tracer

%matplotlib inline

Using TensorFlow backend.


In [2]:
def processNumbers(data):
    cnts = np.zeros(53).astype(int)
    numbers = []
    total_num_entries = 0
    for i,row in data.iterrows():
        for n in row:
            ndx = n-1
            cnts[ndx]+=1 
            total_num_entries+=1
            numbers.append(ndx)
            total_num_entries 
    return np.array(numbers), cnts, total_num_entries

In [3]:
filename = './origdata/florida6.csv'
##["date", "num1", "num2", "num3", "num4","num5","num6"]
data = pd.read_csv(filename,  index_col=False, 
                       header=None,
                       usecols=[1,2,3,4,5,6],
                  )
#print(data[:10])
#numberCounts
numbers, counts, total_entries = processNumbers(data)
print(counts)
print(numbers[:6*6])

[213 183 203 227 209 212 215 217 203 198 201 205 207 187 221 200 191 201
 195 199 208 202 198 195 196 204 199 220 215 197 203 190 196 185 180 217
 207 229 207 205 216 202 205 222 202 194 202 219 200 184 196 229 189]
[16 22 26 34 35 39  8 13 15 19 27 44  0 22 28 30 37 52  5 22 24 35 42 48  6
 13 19 21 26 32  8 11 16 20 29 48]


In [4]:
print(data)

       1   2   3   4   5   6
0     17  23  27  35  36  40
1      9  14  16  20  28  45
2      1  23  29  31  38  53
3      6  23  25  36  43  49
4      7  14  20  22  27  33
5      9  12  17  21  30  49
6     20  21  35  39  49  53
7      4  25  26  29  31  33
8      9  15  21  44  46  47
9      6  17  20  21  40  43
10     1   2   3   4  37  47
11     7  14  16  30  46  48
12    10  12  22  24  31  37
13     4   7  25  34  40  46
14     1   5   9  37  38  53
15     1   6   7   8  43  50
16     7  16  17  27  43  53
17    15  25  43  44  49  53
18    14  25  26  35  41  44
19    10  17  19  30  45  49
20     2   5  28  30  31  33
21    13  21  24  38  47  48
22     2   4  11  14  38  44
23    20  25  37  42  44  48
24    13  18  20  29  48  52
25     2   4   7  21  34  48
26     5   6  24  30  32  42
27     4   5  26  32  37  38
28     8  17  21  41  42  45
29     1  13  20  26  29  31
...   ..  ..  ..  ..  ..  ..
1770  20  12  16  27   2  37
1771  34   5  28  38  12  24
1772   7  18  

##### data

In [4]:
def split_data(nums, batch_size, num_steps, split_frac=0.9):
    slice_size = batch_size * num_steps
    
    #WORKING SOMEWHAT
    #num_rows_to_remove = 2
    #num_ndx_to_remove = num_rows_to_remove * num_steps
    #shift_val = 1
    #n_individualNumbers = len(nums)-num_ndx_to_remove
    #n_rowsOfSizeNumSteps = int(n_individualNumbers / slice_size)
    
    #x = nums[: (n_rowsOfSizeNumSteps*slice_size)]
    #y = nums[shift_val: n_rowsOfSizeNumSteps*slice_size + shift_val]
    
    #num_split_training_rows = int(n_rowsOfSizeNumSteps*split_frac)
    #num_split_idx = (num_split_training_rows*num_steps)
    
    #num_remaining_split_idx = n_individualNumbers - num_split_idx
    
    #num_remaining_rows = (num_remaining_split_idx // num_steps)

    #train_x, train_y= np.stack(np.split(x[:num_split_idx], num_split_training_rows)), np.stack(np.split(y[:num_split_idx], num_split_training_rows))
    #val_x, val_y = np.stack(np.split(x[num_split_idx:],num_remaining_rows)), np.stack(np.split(y[num_split_idx:], num_remaining_rows))
    #END WORKING
    
    n_batches = int(len(nums) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = nums[: n_batches*slice_size-6]
    y = nums[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the first split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    
    print(train_x[:12])

    return train_x, train_y, val_x, val_y
    

In [5]:
train_x, train_y, val_x, val_y = split_data(numbers, 1, 6)

[[16 22 26 ...,  6 45 48]]


### train_x.shape

In [6]:
#train_x[:,:6]
train_x.shape

(1, 9720)

In [7]:
train_y.shape

(1, 9720)

In [8]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    n_batches = int(slice_size/num_steps)

    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [9]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
    
    # When we're using this network for sampling later, we'll be passing in
    # one character at a time, so providing an option for that
    if sampling == True:
        batch_size, num_steps = 1, 1
    
    #Tracer()() #Set breakpoint here
    tf.reset_default_graph()
    
    #Decalre placeholders we'll feed into our graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    #Keep probability placeholder for dropout layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    #One-hot encode the input and target numbers
    x_one_hot = tf.one_hot(inputs, num_classes)
    y_one_hot = tf.one_hot(targets, num_classes)
    
    #print(x_one_hot)
    
    #Build our RNN layers
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    #Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    #Stack up some LSTM layers for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    #Run the data through the RNN layers
    #This makes a list where each element is on step in the sequence
    rnn_inputs = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(x_one_hot, num_steps, 1)]
    
    # Run each sequence step through the RNN and collect the outputs
    outputs, state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=initial_state)
    final_state = state
    
    # Reshape output so it's a bunch of rows, one output row for each step for each batch
    #Tracer()()
    seq_output = tf.concat(outputs, axis=1)
    output = tf.reshape(seq_output, [-1, lstm_size])
    
    # Now connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(num_classes))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and batch
    logits = tf.matmul(output, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    preds = tf.nn.softmax(logits, name='predictions')
    
    # Reshape the targets to match the logits
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    cost = tf.reduce_mean(loss)

    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes
    # NOTE: I'm using a namedtuple here because I think they are cool
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph


In [109]:
#Hyperparams
batch_size = 1
num_steps = 6 
lstm_size = 512
num_layers = 6
learning_rate = 0.001
keep_prob = 0.5

In [111]:
epochs = 9
# Save every N iterations
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(numbers, batch_size, num_steps)

model = build_rnn(len(counts), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    #print("*** ", int(train_x.shape[0]/num_steps))
    n_batches = int(train_x.shape[0]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            print("Shape X: ", x.shape, y.shape)
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer], 
                                                 feed_dict=feed)
 
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/i{}_l{}_v{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

[[16 22 26 ...,  6 45 48]]
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 1/0 Training loss: 3.9703 0.3966 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 2/0 Training loss: 3.9707 0.1512 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 3/0 Training loss: 3.9732 0.1580 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 4/0 Training loss: 3.9743 0.1491 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 5/0 Training loss: 3.9748 0.1617 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 6/0 Training loss: 3.9759 0.1524 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 7/0 Training loss: 3.9751 0.1550 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 8/0 Training loss: 3.9728 0.1619 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 9/0 Training loss: 3.9724 0.1699 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 10/0 Training loss: 3.9679 0.1580 sec/batch
Shape X:  (1, 6) (1, 6)
Epoch 1/9  Iteration 11/0 Training loss: 3.9907 0.1571 sec/batch
Sha

In [112]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i1600_l512_v3.971.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512_v4.014.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512_v3.993.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512_v3.986.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512_v3.977.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512_v3.980.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512_v3.972.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512_v3.981.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512_v3.971.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l512_v3.970.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512_v3.971.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512_v3.977.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512_v3.977.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512_v3.973.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512_v4.000.ckpt"
all_model_checkpoint_paths: "checkpoint

In [113]:
def pick_top_numbers(preds, number_size, top_n=5):
    p = np.squeeze(preds)
    #Length of p is the total number pool we can pull from
    #print(p)
    #print("------")
    #print(np.argsort(p))
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    
    #print(p)
    c = np.random.choice(number_size, 1, p=p)[0]
    return c

In [114]:
def sample(checkpoint, n_samples, lstm_size, number_size, start_num=6):
    samples = [start_num]
    #reduce samples by 1 since we add start_num above
    n_samples = n_samples - 1
    model = build_rnn(number_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    print("Start Num: {}".format(start_num))
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        counter = 0
        max_count = 50
        total_iter = 0
        while counter != n_samples and total_iter != max_count:
            total_iter = temp = total_iter + 1
            #print("Counter {}, total_iter {}".format(counter, total_iter))
            x = np.zeros((1, 1))
            x[0,0] = start_num
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)
            
            c = pick_top_numbers(preds, number_size, 5)+1
            if c not in samples:
                samples.append(c)
                counter = temp2 = counter + 1
            
        
    return samples#''.join(samples)

In [120]:
checkpoint = "checkpoints/i1600_l512_v3.971.ckpt"
samp = sample(checkpoint, 6, lstm_size, len(counts), start_num=randint(1,53))
print(samp)

Start Num: 38
[38, 46, 49, 3, 51, 1]
